# Create resaidual norm files for CREDIT

In [1]:
import os
import yaml
import copy
import numpy as np
import xarray as xr

In [2]:
from scipy.stats import gmean

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## File creation

In [4]:
# get variable information from data_preprocessing/config
config_name = os.path.realpath('data_config_mlevel_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [5]:
N_levels = 46
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg/'

In [6]:
# get variable names
varnames = list(conf['residual'].keys())
varnames = varnames[:-5] # remove save_loc and others

varname_upper = ['specific_humidity', 
                 'specific_total_water', 
                 'temperature', 
                 'u_component_of_wind',
                 'v_component_of_wind']

varname_surf = list(set(varnames) - set(varname_upper))

# collect computed mean and variance values
# See "qsub_STEP01_compute_mean_std.ipynb"
MEAN_values = {}
STD_values = {}

for varname in varname_surf:
    save_name = conf['residual']['save_loc'] + '{}_mean_std_{}.npy'.format(
        conf['residual']['prefix'], varname)
    mean_std = np.load(save_name)
    MEAN_values[varname] = mean_std[0]
    STD_values[varname] = mean_std[1]

for varname in varname_upper:

    # -------------------------------------------- #
    # allocate all levels
    mean_std_all_levels = np.empty((2, N_levels))
    mean_std_all_levels[...] = np.nan
    
    for i_level in range(N_levels):
        save_name = conf['residual']['save_loc'] + '{}_level{}_mean_std_{}.npy'.format(
            conf['residual']['prefix'], i_level, varname)
        mean_std = np.load(save_name)
        mean_std_all_levels[:, i_level] = mean_std

    # -------------------------------------------- #
    # save
    MEAN_values[varname] = np.copy(mean_std_all_levels[0, :])
    STD_values[varname] = np.copy(mean_std_all_levels[1, :])

keys_to_drop = ['TCC', 'MSL', 'SKT', 'specific_humidity', 'land_sea_CI_mask']
MEAN_values = {k: v for k, v in MEAN_values.items() if k not in keys_to_drop}
STD_values = {k: v for k, v in STD_values.items() if k not in keys_to_drop}

In [7]:
std_val_all = list(STD_values.values())
std_val_surf = np.array(std_val_all[:-4])
std_val_upper = std_val_all[-4:]

In [8]:
std_concat = np.concatenate([std_val_surf]+ std_val_upper)
std_g = gmean(np.sqrt(std_concat))

In [9]:
ds_example = xr.open_zarr(base_dir+'all_in_one/ERA5_mlevel_1deg_6h_subset_1979_conserve.zarr')

In [10]:
# ------------------------------------------------------- #
# create xr.DataArray for std
# Initialize level coord
level = np.array(ds_example['level'])

ds_std_6h = xr.Dataset(coords={"level": level})

for varname, data in STD_values.items():
    data = np.sqrt(data) / std_g # <--- var to std and divided by std_g
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_std_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_std_6h[varname] = data_array

In [18]:
ds_std_6h.to_netcdf(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')

In [21]:
base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc'

'/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg/mean_std/residual_6h_1979_2019_conserve_1deg.nc'

In [20]:
# ------------------------------------------------------- #
# Compare with my old ones
base_dir_plevel = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'
new_std = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')
old_std = xr.open_dataset(base_dir_plevel+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')

for varname in varnames:
    try:
        print('=============== {} ================='.format(varname))
        print(np.array(new_std[varname]))
        print(np.array(old_std[varname]))
    except:
        pass

=============== MSL =================
=============== SKT =================
=============== SP =================
0.10132599531217001
=============== TCC =================
=============== VAR_10U =================
1.523816951849432
=============== VAR_10V =================
1.9793276525070973
=============== VAR_2T =================
0.4669417240385481
0.5812322200302392
=============== evaporation =================
2.211363431011042
2.752625456520836
=============== surface_latent_heat_flux =================
2.2148699354345633
2.756990226781677
=============== surface_net_solar_radiation =================
5.019390449726732
6.2479562311557375
=============== surface_net_thermal_radiation =================
2.49941140467327
3.1111771870427525
=============== surface_sensible_heat_flux =================
4.078923303485305
5.077296641030351
=============== toa_incident_solar_radiation =================
5.063505378859014
6.302868904142561
=============== top_net_solar_radiation ================